In [1]:
import pandas as pd

In [2]:
from utils import *

In [3]:
comptes = (
    pd
    .read_csv('data/ratios_inpi_bce.csv', sep=";",dtype={'siren': str})
    .assign(
        siren = lambda df: df.siren.str.pad(9, 'left', '0')
    )
    .query('type_bilan == "C"')
    .assign(
        date_cloture_exercice = lambda df: pd.to_datetime(df.date_cloture_exercice),
        cloture_exercice_annee = lambda df: pd.to_datetime(df.date_cloture_exercice).dt.to_period('y').astype(str)
    )
)

In [4]:
ca = (
    comptes
    .query('siren == "382028199"')
    .pivot_table(
        index = 'siren',
        columns = 'cloture_exercice_annee',
        values = 'chiffre_d_affaires',
        aggfunc= 'last'
    )
)

ca.pipe(format_table)#.pipe(format_index_siren)

cloture_exercice_annee,2016,2017,2018,2019,2020,2021
siren,,,,,,
382028199,38 640 864,36 024 163,34 609 921,31 761 447,24 886 487,27 568 013


In [5]:
aides = (
    pd
    .read_csv('datasets/aides-total_sup_100000-clean.csv')
    .set_index('recodage')
    .loc[['LIBERATION']]
)

aides

,2015,2016,2017,2018,2019,2020,2021,total
recodage,,,,,,,,
LIBERATION,5099910.0,4887544.0,4738019.0,4450177.0,4181332.0,4102802.0,4998785.0,32458569.0


In [6]:
(
    aides
    .div(ca.iloc[0])
    .style
    .pipe(format_percent)
)

,2015,2016,2017,2018,2019,2020,2021,total
recodage,,,,,,,,
LIBERATION,nan%,12.6%,13.2%,12.9%,13.2%,16.5%,18.1%,nan%
